# Political Violence Targeting Women & Demonstrations Featuring Women

#### All violence targeting women, as well as demonstrations featuring women, are included in the data file below. The data in this file cover all events in which women were specifically targeted by political violence, not all events involving women in any way; the file also covers all demonstration events in which women were specifically featured, not all demonstrations involving women. 

#### This dataset was extract from ACLED Access Portal and we perform EDA using Python.


### Data collection

We import pandas and necessary libraries for dataframe manipulation.

In [1]:
import pandas as pd

In [2]:
df_gpv = pd.read_csv("gender_Sep27-1.csv")
display(df_gpv.shape)

(75563, 31)

In [10]:
# First rows are checked
first_rows = df_gpv.head()

In [4]:
# columns's names are checked
columns_name = df_gpv.keys()

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')

In [9]:
# countries are displaying

first_column = df_gpv.iloc[:, 0]

display(first_column)

0        BRA85924
1         ECU3988
2        MEX93498
3        MEX93502
4        YEM92022
           ...   
75558       UGA13
75559       ALG50
75560     SIE4762
75561      RWA652
75562        NIR1
Name: event_id_cnty, Length: 75563, dtype: object